<a href="https://colab.research.google.com/github/hatkiet/Project_4/blob/Mia-branch/Random_Forest_UnderSamplingMH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Import required libraries and dependencies
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

from imblearn.under_sampling import RandomUnderSampler
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report

In [3]:
# Load the data into a Pandas DataFrame
df = pd.read_csv("heart_2022_cleaned.csv")
# Display sample data
df

,Sex,Age,Race,BMI,GenHealth,PhysicalHealth,MentalHealth,PhysicalActivity,ChestScan,SleepHours,...,ECigaretteUsage,Drinking,HeartAttack,Angina,Stroke,Asthma,SkinCancer,Depressed,KidneyDisease,Diabetes
0,0,12,0,26.57,4,0.0,0.0,0,0,6.0,...,0,0,0,0,0,0,0,0,0.0,0.0
1,0,7,0,25.61,3,2.0,3.0,1,0,5.0,...,0,0,0,0,0,0,0,0,0.0,0.0
2,0,4,0,21.77,1,2.0,0.0,1,1,9.0,...,0,1,0,0,0,0,1,0,0.0,0.0
3,1,12,0,26.08,0,1.0,0.0,0,0,7.0,...,0,0,1,0,1,0,1,0,0.0,1.0
4,0,12,1,22.96,3,0.0,0.0,1,0,7.0,...,0,1,0,0,0,0,1,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153891,0,3,0,23.03,2,2.0,30.0,1,0,7.0,...,0,1,0,0,0,0,1,0,0.0,0.0
153892,0,7,0,20.70,2,5.0,3.0,1,0,7.0,...,0,0,1,0,0,1,1,1,0.0,0.0
153893,1,12,0,22.89,1,30.0,0.0,1,1,10.0,...,0,0,0,0,1,0,1,0,0.0,0.0
153894,0,2,0,24.45,2,1.0,3.0,1,0,8.0,...,0,1,0,0,0,0,1,1,0.0,0.0


In [4]:
# Copy dataset for Random Forests - Supervised Learning
df5 = df.copy()

In [5]:
# Define features set X
X = df5.drop("HeartAttack", axis=1)
y = df5["HeartAttack"].values.reshape(-1,1)
y[:5]

array([[0],
       [0],
       [0],
       [1],
       [0]])

In [6]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create an imputer to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

# Fit the imputer on the training data and transform both training and test data
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Create and fit the RandomForestClassifier on the imputed data
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train_imputed, y_train)

In [14]:
rf_model.score(X_train_imputed, y_train)

0.9998700421063575

In [15]:
rf_model.score(X_test_imputed, y_test)

0.9496803035816396

In [16]:
prep = rf_model.predict(X_test_imputed)
prep

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
accuracy_score(y_test, prep)

0.9496803035816396

In [19]:
print(classification_report(y_test, prep))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97     36444
           1       0.56      0.21      0.30      2030

    accuracy                           0.95     38474
   macro avg       0.76      0.60      0.64     38474
weighted avg       0.94      0.95      0.94     38474



In [20]:
df5["HeartAttack"].value_counts()

HeartAttack
0    145724
1      8172
Name: count, dtype: int64

In [23]:
X = df5.drop("HeartAttack",axis=1)
y = df5.HeartAttack
rus = RandomUnderSampler(random_state=78)
X_res, y_res = rus.fit_resample(X, y)

In [24]:
print(y_res.value_counts())

HeartAttack
0    8172
1    8172
Name: count, dtype: int64


In [25]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
# Fitting the model
rf_model = rf_model.fit(X_res, y_res)
print(f'Model score: {rf_model.score(X_res, y_res)}')
prep = rf_model.predict(X_res)

print(f'Accuracy Score: {accuracy_score(y_res, prep)}')
print('')
print(classification_report(y_res, prep))
print('')
print("Confusion Matrix:\n", confusion_matrix(y_res, prep))
print('')

Model score: 0.9999388154674498
Accuracy Score: 0.9999388154674498

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8172
           1       1.00      1.00      1.00      8172

    accuracy                           1.00     16344
   macro avg       1.00      1.00      1.00     16344
weighted avg       1.00      1.00      1.00     16344


Confusion Matrix:
 [[8172    0]
 [   1 8171]]

